In [2]:
import pandas as pd

# Задание 1

In [9]:
visit_log = pd.read_csv('visit_log.csv', sep = ';')
visit_log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [77]:
def source_types(visits_df):
    if visits_df['traffic_source'].lower() in ['yandex', 'google']: return 'organic'
    elif visits_df['traffic_source'].lower() in ['paid', 'email']: 
        if visits_df['region'] == 'Russia': return 'ad'
        else: return 'other'
    else: return visits_df['traffic_source']

In [79]:
visit_log['source_type'] = visit_log.apply(source_types, axis = 1)
visit_log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


# Задание 2

In [82]:
urls = pd.read_csv('urls.txt')
urls.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [92]:
news_urls = urls[ urls.url.str.contains(r'/\d{8}-') ]
news_urls.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


# Задание 3

In [97]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [197]:
def users_lt(user_df):
    if len(user_df) > 100: return user_df['timestamp'].max() - user_df['timestamp'].min()

In [229]:
users_g100 = ratings[ ['userId', 'timestamp'] ].groupby('userId').apply(users_lt).reset_index()
users_g100.columns = ['userId', 'lifetime']
lifetime = users_g100[ ~users_g100['lifetime'].isna() ]['lifetime'].mean()
print(f"Среднее время жизни пользователей, которые выставили более 100 оценок равно {lifetime / 60 / 60 / 24 :.1f} дня.")

Среднее время жизни пользователей, которые выставили более 100 оценок равно 463.9 дня.


# Задание 4

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [234]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [236]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [238]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [240]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [258]:
client_revenues = pd.concat([rzd, auto, air]).groupby('client_id').max().fillna(0)
client_revenues

,rzd_revenue,auto_revenue,air_revenue
client_id,,,
111,1093.0,0.0,0.0
112,2810.0,0.0,0.0
113,10283.0,57483.0,0.0
114,5774.0,83.0,0.0
115,981.0,912.0,81.0
116,0.0,4834.0,4.0
117,0.0,98.0,13.0
118,0.0,0.0,173.0


In [264]:
client_addresses = client_base.merge(right = client_revenues, on = 'client_id', how = 'inner', validate = '1:m')
client_addresses

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0
